
# Classify images using CNNs
###### Based on Jupyter Notebook provided by Dr. Ryan Ahmed @STEMplicity, as part of the Udemy course "Build 8 Practical Projects and Go from Zero to Hero in Deep/Machine Learning, Artificial Neural Networks".

The purpose of this project is not just to follow along with the lesson, but to replicate in my own way how to use CNN to develop a model that can accurately classify images out of the CIFAR-10 dataset.



### 0. Problem Statement

- CIFAR-10 is a dataset that consists of several images divided into the following 10 classes: 
    - Airplanes
    - Cars 
    - Birds
    - Cats
    - Deer
    - Dogs
    - Frogs
    - Horses
    - Ships
    - Trucks

- The dataset stands for the Canadian Institute For Advanced Research (CIFAR)
- CIFAR-10 is widely used for machine learning and computer vision applications. 
- The dataset consists of 60,000 32x32 color images and 6,000 images of each class.
- Images have low resolution (32x32). 
- Data Source: https://www.cs.toronto.edu/~kriz/cifar.html


### 1. Import libraries/Datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn 

### 2. Visualize Data

### 3. Data Preparation

### 4. Model Training

### 5. Model Evaluation

### 6. Saving The Model

### 7. Improving The Model With Data Augmentation

- Image Augmentation is the process of artificially increasing the variations of the images in the datasets by flipping, enlarging, rotating the original images. 
- Augmentations also include shifting and changing the brightness of the images.

#### 7.1. Data Augmentation for the CIFAR-10 dataset

#### 7.2. Model training using augmented dataset

#### 8. References

This tech report (Chapter 3) describes the dataset and the methodology followed when collecting it in much greater detail.

[Learning Multiple Layers of Features from Tiny Images](https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf), Alex Krizhevsky, 2009.